# Fine-tune IndoBERT dengan Huggingface API Trainer
Paper: Bryan Wilie, Karissa Vincentio, Genta Indra Winata, _et al_. IndoNLU: Benchmark and Resources for Evaluating Indonesian Natural Language Understanding. In Proceedings of the 1st Conference of the Asia-Pacific Chapter of the Association for Computational Linguistics and the 10th International Joint Conference on Natural Language Processing.  
Huggingface: https://huggingface.co/indobenchmark/indobert-base-p2

In [1]:
# Optional
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
NUM_LABELS =  3#@param {type:"integer"}
TRAIN_PATH_DATA_INPUT = "/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Data/Non_Stem_Clean/clean_train.csv" #@param {type:"string"}
TEST_PATH_DATA_INPUT = "/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/Final Yuk Bisa Yuk/Simulasi/testing_processed.csv" #@param {type:"string"}

# Install Lib

In [7]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.9 MB 10.1 MB/s 
     |████████████████████████████████| 52 kB 2.0 MB/s 
     |████████████████████████████████| 3.3 MB 32.2 MB/s 
     |████████████████████████████████| 636 kB 59.0 MB/s 
     |████████████████████████████████| 895 kB 53.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 270 kB 6.9 MB/s 
     |████████████████████████████████| 125 kB 73.1 MB/s 
     |████████████████████████████████| 1.3 MB 55.5 MB/s 
     |████████████████████████████████| 243 kB 69.9 MB/s 
     |████████████████████████████████| 294 kB 75.6 MB/s 
     |████████████████████████████████| 142 kB 71.3 MB/s 


# Import Data dan Menyiapkannya
- Label harus dimulai dari 0
- Data text harus di preprocessing default

In [3]:
import pandas as pd
# Training Data Handling
df = pd.read_csv(TRAIN_PATH_DATA_INPUT)

from sklearn.model_selection import train_test_split
train_df, eval_df = train_test_split(df, shuffle=True, test_size=0.2)

train = train_df.drop(columns="raw", axis=1)
train = train_df.rename(columns={
    "processed" : "text",
    "labels" : "label"
})

eval = eval_df.drop(columns="raw", axis=1)
eval = eval_df.rename(columns={
    "processed" : "text",
    "labels" : "label"
})
######### Ini Harusnya ada di Pre-Processing ############
# mapping = {
#     -1 : 0,
#     1 : 1,
# }
# df['label']= df['label'].map(mapping)
# df = df.reset_index(drop=True)
#########################################################



# Testing Data Handling
# df_test = pd.read_csv(TEST_PATH_DATA_INPUT)
# df_test = df_test.drop(columns="raw", axis=1)
# df_test = df_test.rename(columns={
#     "processed" : "text",
#     "labels" : "label"
# })

In [5]:
train.to_csv("./train.csv", index=False)
eval.to_csv("./eval.csv", index=False)
# df_test.to_csv("./test.csv", index=False)

In [9]:
from datasets import load_dataset
files = {"train": "/content/train.csv", 
         "eval": "/content/eval.csv", 
        #  "test":"/content/test.csv"
        }
dataset = load_dataset('csv', data_files=files)

Using custom data configuration default-3256a9a723b55364


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3256a9a723b55364/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

# Tokenizing

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p2", use_fast=True)
tokenizer.add_tokens([ 'NaN' ])

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/224k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

1

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [13]:
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["eval"]

# Modelling

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p2", num_labels=NUM_LABELS)

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Train Preparation

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

# training_args = TrainingArguments(
#     output_dir="./running_trainer",
#     do_train=True,
#     evaluation_strategy="steps",
#     learning_rate=1e-8,
#     num_train_epochs=3,
#     warmup_steps=100,
#     save_strategy="epoch",
#     save_steps=500,
# )

# Tarining

In [17]:
from transformers import Trainer
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
train_history = trainer.train()
train_evaluate = trainer.evaluate()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: raw, text.
***** Running training *****
  Num examples = 7850
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2946


Step,Training Loss
500,0.701300
1000,0.613400
1500,0.454200
2000,0.441700
2500,0.301800


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin


Training completed. Do not forget to share your mod

In [18]:
train_evaluate

{'epoch': 3.0,
 'eval_accuracy': 0.7890983188996434,
 'eval_loss': 0.8942208886146545,
 'eval_runtime': 17.6985,
 'eval_samples_per_second': 110.913,
 'eval_steps_per_second': 13.899}

# Model Saving

In [19]:
trainer.save_model("/content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/IndoBERTUnbalanceNonStemmed 0.79")

Saving model checkpoint to /content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/IndoBERTUnbalanceNonStemmed 0.79
Configuration saved in /content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/IndoBERTUnbalanceNonStemmed 0.79/config.json
Model weights saved in /content/drive/Shareddrives/DAC UNPAD: Yaudahlah/FINAL MENANG BISMILLAH/Model/IndoBERTUnbalanceNonStemmed 0.79/pytorch_model.bin
